In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
db = pd.read_csv("/Users/damoncrockett/Dropbox/lml/export_08_26_24.csv", encoding='latin-1')

In [3]:
len(db)

7245

In [3]:
db = db.loc[db.Omit_from_Paperbase==0]

In [4]:
db.Format.value_counts()

Sample book                     4509
Not exposed - package open      1539
Not exposed - package sealed     800
Package only                     123
Exposed - package open            47
Name: Format, dtype: int64

In [5]:
keepcols = [
    'Year','DateUncertain','Catalog Number','Secondary Catalog Number','Manufacturer','Brand',
    'Texture2', 'Reflectance2', 'BaseColor2', 'Weight2','SurfaceDesignation2',
    'Exposure','BackprintingPrimary','Postcards','Toner','Resin_coated?','ProcessName','Format','Size'
]

In [6]:
db = db[keepcols]
db = db.fillna('')
db['catalog'] = db['Catalog Number'].apply(str) + db['Secondary Catalog Number']

In [7]:
db.columns = [
    'year','circa','cat','scat','man','bran','xword','gword','cword','tword','surf','expos',
    'backp','postcard','toner','resin','photoprocess','forma','dims','catalog'
]

In [8]:
db = db[['forma','catalog','cat','scat','man','bran','year','circa','surf','xword','gword','cword','tword','expos',
    'backp','postcard','toner','resin','photoprocess','dims']]

In [9]:
db.photoprocess = db.photoprocess.apply(lambda x: x.strip('\t'))

In [10]:
db = db.loc[db.photoprocess=='gelatin silver process']

In [11]:
del db['photoprocess']

In [12]:
db = db.reset_index(drop=True)

In [13]:
db.catalog.loc[db.catalog=='2631yy'] = ['2631xx','2631yy']
db.cat = db.cat.apply(str)
db.scat.loc[(db.cat=='2631')&(db.scat=='yy')] = ['xx','yy']

In [14]:
db.year.loc[db.cat=='5412'] = 1957

In [15]:
db.catalog = db.catalog.apply(lambda x: x.lower())

In [16]:
db.loc[len(db)] = pd.Series(['5616','Gevaert','[Indeterminate]','1950','5616','',1,
                            '[not specified]','[texture unspecified]','[gloss unspecified]',
                             '[base color unspecified]','[weight unspecified]',
                            '','',0,'',0,'Not exposed - package open',''],
                            index=['catalog','man','bran','year','cat','scat','circa',
                                  'surf','xword','gword','cword','tword',
                                  'expos','backp','postcard','toner','resin','forma','dims'])

In [17]:
db.loc[len(db)] = pd.Series(['286z','Ansco','Scona Reflex','1955',
                             'Single Weight','Semi Matte','Smooth','286','z',1,'[not specified]',
                             '[base color unspecified]',
                            '','',0,'',0,'Sample book',''],
                            index=['catalog','man','bran','year','tword','gword',
                                   'xword','cat','scat','circa','surf','cword',
                                  'expos','backp','postcard','toner','resin','forma','dims'])

In [18]:
db.catalog.loc[db.catalog.str.contains('_')] = db.catalog.loc[db.catalog.str.contains('_')].apply(lambda x: x.replace('_',''))

In [19]:
db.expos.loc[db.catalog.isin(['5617','5618','5619'])] = ''

# sample type

In [20]:
iiif = pd.read_pickle("/Users/damoncrockett/lml2lux/notebooks/db.pkl")

In [21]:
iiif.columns

Index(['PhotoID', 'Year', 'DateUncertain', 'barecat',
       'Secondary Catalog Number', 'Manufacturer', 'Brand', 'Format',
       'Texture2', 'Reflectance2', 'BaseColor2', 'Weight2',
       'SurfaceDesignation2', 'Omit_from_Paperbase', 'LocationBox',
       'LocationBag', 'Exposure', 'BackprintingPrimary', 'Postcards', 'Toner',
       'Resin_coated?', 'ProcessName', 'Link', 'AATA_ID', 'catalog', 'sb',
       'imfile', 'fbase', 'imfiles', 'imfiles_type', 'imfiles_md5'],
      dtype='object')

In [22]:
iiif = iiif[['catalog','sb','imfiles']]

In [23]:
iiif.catalog.loc[iiif.catalog.str.contains('_')] = iiif.catalog.loc[iiif.catalog.str.contains('_')].apply(lambda x: x.replace("_",""))

In [24]:
db = db.join(iiif.set_index('catalog'),'catalog')

In [25]:
db.sb.loc[db.catalog=='286z'] = True

In [26]:
db.forma.loc[db.catalog.isin(['4148','4861'])] = "unknown"

In [27]:
db.sb.loc[db.catalog.isin(['114','5616','5617','5618','5619'])] = False

In [28]:
db.bran.loc[db.catalog=='17l'] = 'Rekord Rapid'

In [29]:
db.loc[len(db)] = pd.Series(['17q','Agfa Gevaert','Portriga Rapid','1974','17','q',0,
                            '138','[texture unspecified]','[gloss unspecified]',
                             '[base color unspecified]','[weight unspecified]',
                            21.0,'A10',0,'',0,True,'Sample book','',['17_021.jpg']],
                            index=['catalog','man','bran','year','cat','scat','circa',
                                  'surf','xword','gword','cword','tword',
                                  'expos','backp','postcard','toner','resin','sb','forma','dims','imfiles'])

In [30]:
db.imfiles.loc[db.imfiles.isnull()] = ''

In [31]:
def makelist(e):
    if isinstance(e,list):
        return e
    elif isinstance(e,str):
        return [e]

In [32]:
db.imfiles = db.imfiles.apply(makelist)

In [33]:
def get_sbid(db,i):
    sb = db.sb.loc[i]
    if sb:
        return db.cat.loc[i]
    else:
        return ''

In [34]:
def get_paper_type(db,i):
    man = db.man.loc[i]
    bran = db.bran.loc[i]
    year = db.year.loc[i]
    surf = db.surf.loc[i]
    xword = db.xword.loc[i]
    gword = db.gword.loc[i]
    cword = db.cword.loc[i]
    tword = db.tword.loc[i]
    
    return f'{man} {bran} {year} {surf} {xword} {gword} {cword} {tword}'

In [35]:
db.expos.loc[db.sb==False] = ''

In [36]:
db['sbid'] = [get_sbid(db,i) for i in db.index]

In [37]:
db['pid'] = [get_paper_type(db,i) for i in db.index]

In [38]:
db = db.sort_values(['cat','scat'])
db = db.reset_index(drop=True)

In [39]:
db.sb = db.sb.apply(int)

In [40]:
db.sb.loc[db.cat=='3194'] = 1
db.sb.loc[db.cat=='5440'] = 1

# Images

In [41]:
from PIL import Image

In [42]:
# for i in db.index:
#     sb = db.sb.loc[i]
#     imfiles = db.imfiles.loc[i]
#     for imfile in imfiles:
#         try:
#             if sb:
#                 Image.open(f'/Users/damoncrockett/paperbase/src/assets/img/samplebooks/{imfile}')
#             else:
#                 Image.open(f'/Users/damoncrockett/paperbase/src/assets/img/packages/{imfile}')
#         except:
#             print(i,imfiles)

#### samplebooks

We want an image for every sample, and this means there will be duplicate images with different filenames.

In [43]:
import shutil

In [44]:
# for i in db.index:
#     sb = db.sb.loc[i]
#     catalog = db.catalog.loc[i]
#     if sb:
#         imfiles = db.imfiles.loc[i]
#         imfile = imfiles[0] # only one case of multiple images, verso of a single sample print
#         imfilepath = f'/Users/damoncrockett/paperbase/src/assets/img/samplebooks_expos/{imfile}'
#         try:
#             Image.open(imfilepath)
#             newimfile = f'{catalog}.jpg'
#             shutil.copy(imfilepath,f'/Users/damoncrockett/paperbase/src/assets/img/samplebooks/{newimfile}')
#         except:
#             print(catalog)

#### packages

In [45]:
import os

We just want a simple list of the suffixes, if there are any.

In [46]:
suffstrs = []
for i in db.index:
    sb = db.sb.loc[i]
    catalog = db.catalog.loc[i]
    if sb:
        suffstrs.append('')
        continue
    else:
        imfiles = db.imfiles.loc[i]
        suffs = []
        for imfile in imfiles:
            fbase = os.path.splitext(imfile)[0]
            for j,c in enumerate(fbase):
                if c.isdigit():
                    continue
                else:
                    suff = fbase[j:]
                    suffs.append(suff)
                    break
        if len(suffs) > 0:
            suffstr = '|'.join([item for item in suffs])
        else:
            suffstr = ''
    suffstrs.append(suffstr)

In [47]:
db['suffs'] = suffstrs

In [48]:
db.to_pickle('/Users/damoncrockett/lmlproc/db.pkl')

In [49]:
# DO
# all image assets f'{dir}/{catalog}.jpg' [pkg,sb,backp,procinstruct,texture]
# sync LUX stuff
# sample_type
# decide whether to eliminate more rows
# sample book identity (combo of cat and sample_type, I guess)
# man/bran/year/surface groups